In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

# AC / Lake

In [ ]:
# itp_id = 4 ## AC Transit
itp_id = 159 ## Lake Transit

In [ ]:
import numpy as np

In [ ]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [ ]:
date_range

# Generate analysis data for all dates

In [ ]:
pbar = tqdm()

In [ ]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
#         rt_analysis.export_views_gcs()
#         print(f'complete for date: {date}')
#     except Exception as e:
#         print(e)
#         print(f'failed for date: {date}')
#         continue

In [ ]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 4))

In [ ]:
# corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson') ## alameda
corridor = gpd.read_file('./corridors_sccp_lpp/lakeport_map.geojson') ## lakeport

In [ ]:
rt_day.add_corridor(corridor)

In [ ]:
mappable_stops = (rt_day.stop_delay_view
          >> distinct(_.shape_id, _.stop_sequence, _keep_all=True)
          >> filter(_.corridor)
          >> select(_.stop_id, _.geometry, _.stop_sequence)
         )

In [ ]:
# quick corridor map
# rt_day.quick_map_corridor()

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops
* Assert that more than 10% of filtered stops within corridor, if not then drop shape (remove shapes starting/ending in corridor but mostly running outside)

# Report:

* Sum of median trip delays
* Median segment speed
* Delay compared to speed standards: 16mph (approx. 75th %ile for medium size operator by 2010 NTD)

# Corridor-based metrics for SCCP

* intent is to eventually parameterize this and output formatted report, also methodology may change

In [ ]:
pbar = tqdm()

In [ ]:
rt_day.corridor_metrics()

In [ ]:
schedule_metrics = []
speed_metrics = []
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
        rt_day.add_corridor(corridor)
        _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
        metrics = rt_day.corridor_metrics()
        schedule_metrics += [metrics['schedule_metric_minutes']]
        speed_metrics += [metrics['speed_metric_minutes']]
        print(f'complete for date: {date}')
    except Exception as e:
        print(f'failed for date: {date}')
        print(e)
        continue

In [ ]:
np.round(np.mean(schedule_metrics), 0)

In [ ]:
np.round(np.mean(speed_metrics), 0)

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 4), pbar)
rt_day.add_corridor(corridor)
_m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

In [ ]:
_m.save('lake_map.html')

In [ ]:
rt_day.stop_delay_view >> filter(_.corridor) >> distinct(_.route_short_name)

# Samtrans

In [ ]:
itp_id = 290 # Samtrans

In [ ]:
import numpy as np

In [ ]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [ ]:
date_range

# Generate analysis data for all dates

In [ ]:
pbar = tqdm()

In [ ]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
        rt_analysis.export_views_gcs()
        print(f'complete for date: {date}')
    except Exception as e:
        print(e)
        print(f'failed for date: {date}')
        continue

In [ ]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

In [ ]:
importlib.reload(rt_filter_map_plot)

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 5))

In [ ]:
corridor = gpd.read_file('./corridors_sccp_lpp/peninsula.geojson') ## foster city / hayward park

In [ ]:
rt_day.add_corridor(corridor)

In [ ]:
# quick corridor map
rt_day.quick_map_corridor()

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops

# Report:

* Sum of median trip delays
* Median segment speed
* Delay compared to speed standards: 16mph (approx. 75th %ile for medium size operator by 2010 NTD)

# Corridor-based metrics for SCCP

* intent is to eventually parameterize this and output formatted report, also methodology may change

In [ ]:
pbar = tqdm()

In [ ]:
rt_day.corridor_metrics()

In [ ]:
schedule_metrics = []
speed_metrics = []
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
        rt_day.add_corridor(corridor)
        _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
        metrics = rt_day.corridor_metrics()
        schedule_metrics += [metrics['schedule_metric_minutes']]
        speed_metrics += [metrics['speed_metric_minutes']]
        print(f'complete for date: {date}')
    except Exception as e:
        print(f'failed for date: {date}')
        print(e)
        continue

In [ ]:
np.round(np.mean(schedule_metrics), 0)

In [ ]:
np.round(np.mean(speed_metrics), 0)

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 5), pbar)
rt_day.add_corridor(corridor)
_m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

In [ ]:
_m

In [ ]:
_m.save('samtrans_map.html')

In [ ]:
rt_day.stop_delay_view >> filter(_.corridor) >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_stop_delays